In [46]:
import agents
from agents import Agent,function_tool,Runner

In [3]:
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIAgentsSDKRunner

chat_interface = IPythonChatInterface()

In [4]:
runner = Runner()

### summary agent

In [5]:
import youtube

In [6]:
def fetch_youtube_transcript(video_id: str) -> str:
    """
    Fetches the transcript of a YouTube video and converts it into a subtitle-formatted string.

    Args:
        video_id (str): The unique YouTube video ID.

    Returns:
        str: The subtitles generated from the video's transcript.
    """
    return youtube.fetch_transcript_cached(video_id)

In [7]:
summary_instructions = """
You're a helpful assistant that summarizes youtube videos
"""

In [12]:
summary_agent = Agent(
    name ='summary_agent',
    instructions = summary_instructions,
    handoff_description="whenever the user needs a summary of the video",
    tools=[function_tool(fetch_youtube_transcript)],
    model='gpt-4o-mini'
)

### search agent

In [13]:
from minsearch import AppendableIndex

In [14]:
index = AppendableIndex(text_fields=['content'])

In [32]:
import docs
from typing import Any, Dict

In [36]:
# class Search_Tools:
#     def __init__(self,index):
#         self.index=index
#     def index_video(self,video_id) -> bool :
#         content = youtube.fetch_transcript_cached(video_id)
#         chunks = docs.sliding_window(content,2000,1000)
#         for chunk in chunks:
#             chunk['video_id'] = video_id
#             self.index.append(chunk)
#         return True
#     def search( self,query) -> dict[str,Any]:
#         return self.index.search(query,num_results=5)
    

In [39]:
searchtools = Search_Tools(index)

In [40]:
searchtools.index_video('2ZOnA19sDpM')

True

In [41]:
searchtools.search('sagemaker')

[{'start': 0,
  'content': "0:00 First of all, thank you very much for uh\n0:02 for joining the event. This is the first\n0:04 event we ever have. Uh so this is a new\n0:07 experience for us as well. So I hope\n0:10 everything um goes smoothly\n0:13 and this is the first event but not the\n0:16 last one. We already have uh a few\n0:18 events planned. So the first one, this\n0:20 one is about deploying models with\n0:22 SageMaker. But u we want to meet every\n0:26 Tuesday\n0:27 at u 5:00 p.m. uh European time to talk\n0:31 about different uh technical topics. So\n0:33 the next week we will have a topic about\n0:35 customer segmentation. Then we will have\n0:37 a topic about uh fighting uh fraud uh\n0:41 with triplet loss about neural nets and\n0:43 then um slightly different topic more on\n0:46 the soft side of skills about\n0:49 communication skills for um data\n0:52 scientists and other data professionals.\n0:56 In addition to that, we also want to\n0:57 have something uh like a light

In [42]:
# for above code add hints and doc strings
from typing import Any, Dict, List, Optional
from requests.exceptions import RequestException

class SearchTools:

    def __init__(self, index: Any) -> None:
        self.index = index

    def index_video(self, video_id: str, content: Optional[str] = None) -> bool:
        """
        Fetch and index the transcript of a YouTube video.

        Args:
            video_id (str): The unique identifier of the YouTube video.

        Returns:
            bool: True if indexing succeeds, False otherwise.
        """
        try:
            if not content:
                content = youtube.fetch_transcript_cached(video_id)
                if not content:
                    print(f"No subtitles found for video: {video_id}")
                    return False

            chunks = docs.sliding_window(content, 2000, 1000)
            for chunk in chunks:
                chunk["video_id"] = video_id
                self.index.append(chunk)

            return True

        except (RequestException, ValueError) as e:
            print(f"Error indexing video {video_id}: {e}")
            return False
        except AttributeError:
            print("Error: 'youtube' or 'docs' module not properly configured.")
            return False
        except Exception as e:
            print(f"Unexpected error during indexing: {e}")
            return False

    def search(self, query: str) -> Dict[str, Any]:
        """
        Search for relevant results in the indexed video transcripts.

        Args:
            query (str): The user's search query.

        Returns:
            Dict[str, Any]: A dictionary containing the top search results.
        """
        if not isinstance(query, str) or not query.strip():
            raise ValueError("Query must be a non-empty string.")

        try:
            results = self.index.search(query, num_results=5)
            return results
        except AttributeError:
            raise AttributeError("Index object must implement a 'search(query, num_results=...)' method.")
        except Exception as e:
            print(f"Error performing search: {e}")
            return {}


In [43]:
search_tools = SearchTools(index)

In [50]:
# from toyaikit.tools import wrap_instance_methods

In [ ]:
#search_method_tools = wrap_instance_methods(function_tool, search_tools)

In [48]:
search_method_tools = [
    function_tool(search_tools.index_video),
    function_tool(search_tools.search)
]                

In [51]:
search_instructions = """
Your task is to search through indexed documents.

Before performing a search:

1. Check if the document has been indexed.
2. If not, call `index_video` to index it.

Important rules:

- Do NOT call `search` using a YouTube ID, webpage URL, or any direct link as the query.
- After successfully indexing a source, do NOT perform a search automatically.
- Instead, tell the user something like:
  "The video (or page) has been indexed. What would you like to learn about it?"

Only perform a search if the user provides a natural-language question or topic of interest
(e.g., "What does the video say about AI safety?"), not a URL or ID.

Do not attempt to summarize content.
"""

search_agent = Agent(
    name='search_agent',
    instructions=summary_instructions,
    handoff_description="Whenever the user needs to search for things about videos",
    tools=search_method_tools,
    model='gpt-4o-mini'
)

In [52]:
runner= OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=search_agent
)

In [53]:
await runner.run();

You: how sagemaker will execute docker


You: i want to ask about this video_id - 2ZOnA19sDpM


You: stop


Chat ended.


### triage / orchestrator agent

In [55]:
from agents import handoff

In [57]:
triage_instructions = """
You are the orchestrator between two specialized agents:

1. summarizing_agent — summarizes web pages or YouTube videos.
2. search_agent — searches within previously indexed documents to answer detailed or follow-up questions.

Routing rules:

- If the user sends a YouTube link without any question → hand off to summary_agent.
- If the user asks for a summary, overview, or "what is this video about" → summary_agent.
- If the user asks a follow-up question after a summary (e.g. "how did they do X?", "what does she say about Y?"),
  or refers to something mentioned in a previously summarized or indexed resource → hand off to search_agent.
- If the user asks a direct content question about a topic inside a resource → search_agent.
- Prefer delegating the answer to agents when possible. 

Examples:
User: "https://youtube.com/watch?v=abc123"
→ summary_agent

User: "What does the video say about climate change?"
→ search_agent

User: "How exactly did they fight malaria?" (after a summary)
→ search_agent
""".strip()
triage_agent = Agent(
    name ='triage_agent',
    instructions = triage_instructions,
    handoffs= [
        handoff(summary_agent, on_handoff=lambda ctx: print('handoff to summary agent')),
        handoff(search_agent, on_handoff=lambda ctx: print('handoff to search agent')),
    ],
    model='gpt-4o-mini'
)

In [58]:
runner = OpenAIAgentsSDKRunner(
    chat_interface=chat_interface,
    agent=triage_agent
)

await runner.run();

You: i want to know about this video id - 2ZOnA19sDpM


handoff to summary agent
handoff: transfer_to_summary_agent
handoff: summary_agent -> triage_agent successful


You: i want to know more about how api gateway in sagemaker 


handoff to search agent
handoff: transfer_to_search_agent
handoff: search_agent -> triage_agent successful


You: stop


Chat ended.
